In [1]:
import math
import time
import random
import copy

## Problema Dormitórios

In [2]:
dormitorios = ['sao paulo', 'flamengo', 'coritiba', 'cruzeiro', 'fortaleza']

preferencias = {
    'amanda' : ('cruzeiro', 'coritiba'),
    'pedro' : ('sao paulo', 'fortaleza'),
    'marcos' : ('flamengo', 'sao paulo'),
    'priscila' : ('sao paulo', 'fortaleza'),
    'jessica' : ('flamengo', 'cruzeiro'),
    'paulo' : ('coritiba', 'fortaleza'),
    'fred' : ('fortaleza', 'flamengo'),
    'suzana' : ('cruzeiro', 'coritiba'),
    'laura' : ('cruzeiro', 'coritiba'),
    'ricardo' : ('coritiba', 'flamengo')
}

In [3]:
dominio = [(0, (len(dormitorios) * 2) - i - 1) for i in range(len(dormitorios) * 2)]
dominio

[(0, 9),
 (0, 8),
 (0, 7),
 (0, 6),
 (0, 5),
 (0, 4),
 (0, 3),
 (0, 2),
 (0, 1),
 (0, 0)]

In [4]:
def print_vagas(solucao:list):
    vagas = []

    for i in range(len(dormitorios)):
        # deminstrar que para cada quarto haverá duas vagas
        # ex: 0, 0, 1, 1. Indo pelo indice dos dormitorios
        vagas += [i, i]

    for i in range(len(solucao)):
        # pega o numero do indice do dormitorio
        dormitorio = dormitorios[vagas[solucao[i]]]
        print(list(preferencias.keys())[i], dormitorio)
        # apos preencher a vaga, retira ela
        # limpa a lista de vagas
        del vagas[solucao[i]]

print_vagas(solucao=[6,1,2,1,2,0,2,2,0,0])

amanda cruzeiro
pedro sao paulo
marcos flamengo
priscila flamengo
jessica coritiba
paulo sao paulo
fred fortaleza
suzana fortaleza
laura coritiba
ricardo cruzeiro


In [5]:
def custo_dormitorio(solucao:list):
    vagas = []
    custo = 0

    for i in range(len(dormitorios)):
        # deminstrar que para cada quarto haverá duas vagas
        # ex: 0, 0, 1, 1. Indo pelo indice dos dormitorios
        vagas += [i, i]

    for i in range(len(solucao)):
        # pega o dormitorio na posicao i
        dormitorio = dormitorios[vagas[solucao[i]]]
        # recupera as preferencias da pessoa atual na posicao i
        pref = preferencias.get(list(preferencias.keys())[i])
        
        # primeiro quarto desejado
        if dormitorio == pref[0]:
            pass
        # segunda preferencia + 1 pen
        elif dormitorio == pref[1]:
            custo += 1
        # nenhuma das duas + 3 pen
        else:
            custo += 3

        del vagas[solucao[i]]

    return custo


custo_dormitorio(solucao=[6,1,2,1,2,0,2,2,0,0])

16

## Busca randômica

In [14]:
def busca_randomica(dominio, funcao_custo, numero_solucao):
    melhor_custo = 999
    melhor_solucao = None

    for i in range(numero_solucao):
        # gera uma solucao randomica
        solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]

        # calcula os custos
        custo_solucao = funcao_custo(solucao)

        # verifica se a solucao foi melhor
        if custo_solucao < melhor_custo:
            melhor_custo = custo_solucao
            melhor_solucao = solucao

    return melhor_solucao

custo_dormitorio(busca_randomica(dominio=dominio, funcao_custo=custo_dormitorio, numero_solucao=1000))


6

## Hill Climb

In [24]:
def hill_climb(dominio, funcao_custo):
    # gera uma solucao randomica
    solucao = busca_randomica(dominio=dominio, funcao_custo=funcao_custo, numero_solucao=1000)

    while True:
        vizinhos = []

        for i in range(len(dominio)):
            # verifica se o numero é menor que o valor maximo
            if solucao[i] < dominio[i][1]:
                # copia a solucao
                novo_vizinho = copy.deepcopy(solucao)
                # modifica o vizinho na posicao i em +1
                novo_vizinho[i] = novo_vizinho[i] + 1
                vizinhos.append(novo_vizinho)
            
            # verifica se o valor é maior que o menor valor
            if solucao[i] > dominio[i][0]:
                novo_vizinho = copy.deepcopy(solucao)
                # modifica o vizinho na posicao i em -1
                novo_vizinho[i] = novo_vizinho[i] - 1

        # verifica o custo da soluca
        atual = funcao_custo(solucao)
        # adiciona 
        melhor = atual

        for i in range(len(vizinhos)):
            custo = funcao_custo(vizinhos[i])

            # substitui com as solucoes melhores
            if custo < melhor:
                melhor = custo
                solucao = vizinhos[i]

        # melhor solucao achada
        if melhor == atual:
            break

    return solucao

custo_dormitorio(hill_climb(dominio=dominio, funcao_custo=custo_dormitorio))

5